Welcome to this notebook. We'll explore the ATP tennis data records.

The goal is to try to earn money while bidding online. 
If the end model is predicting winning probabilities that are correct, then we can go to a bidding website and compare the «quotes» on the website to the winning probability predicted by the model. And then we bet against any incorrect «quotes». 

Bidders online may underestimate a player's odds, due to a variety of bias in which case we'd bid on him. 

First let's open the CSV, and look a bit a the data

In [105]:
import pandas
import numpy as np
import matplotlib
import sklearn

In [106]:

tennis_atp_records = pandas.read_csv('ATP.csv')  #, dtype={"draw_size": numpy.int32}, low_memory=False) 
# i had an error due to various datatypes in the  same column. may it's due to the empty values, NaN. low_memory makes it dynamic
# the problem was actually a incorrect csv. columns headers where included twice,
# the csv was build from copy pasting the text from 2 csv together, without removing the second csv's columns names

/home/francois/CasinoRoyal/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (9,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [107]:
tennis_atp_records.tail()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
163960,2017-M-DC-2017-WG-M-RUS-SRB-01,Davis Cup WG R1: RUS vs SRB,Hard,4,D,20170203,5,106421,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163961,2017-M-DC-2017-WG-M-SUI-USA-01,Davis Cup WG R1: SUI vs USA,Hard,4,D,20170203,1,106058,NaN,NaN,...,4.0,4.0,5.0,82.0,52.0,38.0,9.0,13.0,7.0,12.0
163962,2017-M-DC-2017-WG-M-SUI-USA-01,Davis Cup WG R1: SUI vs USA,Hard,4,D,20170203,2,104545,NaN,NaN,...,9.0,7.0,9.0,113.0,76.0,53.0,17.0,19.0,7.0,11.0
163963,2017-M-DC-2017-WG-M-SUI-USA-01,Davis Cup WG R1: SUI vs USA,Hard,4,D,20170203,4,105023,NaN,NaN,...,0.0,3.0,2.0,76.0,42.0,30.0,14.0,10.0,8.0,10.0
163964,2017-M-DC-2017-WG-M-SUI-USA-01,Davis Cup WG R1: SUI vs USA,Hard,4,D,20170203,5,105449,NaN,NaN,...,4.0,1.0,2.0,54.0,27.0,15.0,15.0,9.0,2.0,6.0


For the Davis Cup, there seems to be as many tournaments as there were country-vs-country matches. Let's see if any other tournament has a 4 player pool size

In [108]:
no_davis = tennis_atp_records[-tennis_atp_records['tourney_name'].apply(lambda X: X.startswith('Davis Cup'))] 
no_davis[no_davis['draw_size'] ==4].loc[:,['tourney_name', 'tourney_date', 'winner_name', 'loser_name']]

,tourney_name,tourney_date,winner_name,loser_name
23469,Pepsi Grand Slam,19760710,Ilie Nastase,Manuel Orantes
23737,WCT Challenge Cup,19761202,Jimmy Connors,Vitas Gerulaitis
23738,WCT Challenge Cup,19761202,Ilie Nastase,Manuel Orantes
23739,WCT Challenge Cup,19761202,Ilie Nastase,Jimmy Connors
29480,Pepsi Grand Slam,19770711,Bjorn Borg,Adriano Panatta
29481,Pepsi Grand Slam,19770711,Jimmy Connors,Manuel Orantes
29482,Pepsi Grand Slam,19770711,Bjorn Borg,Jimmy Connors
34173,Pepsi Grand Slam,19790210,Jimmy Connors,John Mcenroe
34174,Pepsi Grand Slam,19790210,Bjorn Borg,Guillermo Vilas
34175,Pepsi Grand Slam,19790210,Bjorn Borg,Jimmy Connors


It seems like only 16 games where played in «4 players» tournaments outside of the Davis Cup, which are not exactly 4 players tournaments.



In [116]:
tennis_atp_records['winner_name'].isnull()

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
163935    False
163936    False
163937    False
163938    False
163939    False
163940    False
163941    False
163942    False
163943    False
163944    False
163945    False
163946    False
163947    False
163948    False
163949    False
163950    False
163951    False
163952    False
163953    False
163954    False
163955    False
163956    False
163957    False
163958    False
163959    False
163960    False
163961    False
163962    False
163963    False
163964    False
Name: winner_name, Lengt

In [117]:
tennis_atp_records['winner_name'].isnull().where(tennis_atp_records['winner_name'].isnull())

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
26       NaN
27       NaN
28       NaN
29       NaN
          ..
163935   NaN
163936   NaN
163937   NaN
163938   NaN
163939   NaN
163940   NaN
163941   NaN
163942   NaN
163943   NaN
163944   NaN
163945   NaN
163946   NaN
163947   NaN
163948   NaN
163949   NaN
163950   NaN
163951   NaN
163952   NaN
163953   NaN
163954   NaN
163955   NaN
163956   NaN
163957   NaN
163958   NaN
163959   NaN
163960   NaN
163961   NaN
163962   NaN
163963   NaN
163964   NaN
Name: winner_name, Length: 163965, dtype: float64

In [103]:
tennis_atp_records.where(tennis_atp_records.loc[:,['winner_name']]=='Jean michel Dupoux Marco')

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
